import the stock list excel file using pandas

In [107]:
import pandas as pd

stocks_list = pd.read_excel("stockslist.xlsx")
max_row = len(stocks_list.index)
# print(str(stocks_list["Holder"][0]))
stocks_list

,Stocks,Holder,Qty,PurchaseDate,PurchasePrice,Expectation
0,RCOM.BO,Ridhi,100,2021-05-20,1.75,300


Initiate our data frame with column names

In [165]:

my_columns = ["holder","qty","stock","purchase_price  ","current_price ","per_gain_loss (%)","per_growth (%)","annual_return (%)","tot_gain_loss ","volume (lk)","mark_cap (cr)","week52high ","week52low ","purchase_date","expectance"]

final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,holder,qty,stock,purchase_price,current_price,per_gain_loss (%),per_growth (%),annual_return (%),tot_gain_loss,volume (lk),mark_cap (cr),week52high,week52low,purchase_date,expectance


Lets get the details about the stock from the API calls

In [167]:
import yfinance as yf

stock = yf.Ticker("RCOM.BO")
stock.info

{'previousClose': 3.09,
 'regularMarketOpen': 3.24,
 'twoHundredDayAverage': 2.0068383,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 3.24,
 'navPrice': None,
 'averageDailyVolume10Day': 4905559,
 'totalAssets': None,
 'regularMarketPreviousClose': 3.09,
 'fiftyDayAverage': 1.9452941,
 'trailingAnnualDividendRate': None,
 'open': 3.24,
 'toCurrency': None,
 'averageVolume10days': 4905559,
 'expireDate': None,
 'yield': None,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'regularMarketDayLow': 3.12,
 'priceHint': 4,
 'currency': 'INR',
 'regularMarketVolume': 43749321,
 'lastMarket': None,
 'maxSupply': None,
 'openInterest': None,
 'marketCap': 8891370496,
 'volumeAllCurrencies': None,
 'strikePrice': None,
 'averageVolume': 8389232,
 'priceToSalesTrailing12Months': None,
 'dayLow': 3.12,
 'ask': 0,
 'ytdReturn': None,
 'askSize': 0,
 'volume

In [168]:
x = datetime.datetime.now()

In [102]:
def annual_return(pd,pp,cp):
    cd = datetime.datetime.now()
    gain_loss = (cp-pp)/pp*100
    days = (cd-pd).days
    
    if days/365>1:
        return round(gain_loss/(days/365),2)
    else:
        return round(gain_loss,2)
    
def gain_loss(pp,cp):
    return format((cp-pp)/pp*100,'.2f')

def growth(pp,cp):
    return format(cp/pp*100,'.2f')

In [70]:
print("cap - "+ str(stock.info["marketCap"])+ 
      "| vol - " + str(stock.info["volume"]) + 
      "| w52h - " + str(stock.info["fiftyTwoWeekHigh"])+ 
      "| w52l - " + str(stock.info["fiftyTwoWeekLow"]) + 
      "| regp - " + str(stock.info["regularMarketPrice"]) +
      "| prevc - "+ str(stock.info["previousClose"])
     )
# cap - 8891370496| vol - 41924106| w52h - 4.33| w52l - 0.62| regp - 3.24| prevc - 3.09

cap - 8891370496| vol - 42989107| w52h - 4.33| w52l - 0.62| regp - 3.24| prevc - 3.09


In [166]:
import yfinance as yf

for row in range(0,max_row):
    ticker = stocks_list["Stocks"][row]
    stock = yf.Ticker(ticker)
#     print(stock.info)
    final_dataframe = final_dataframe.append(
        pd.Series([
            stocks_list["Holder"][row],
            stocks_list["Qty"][row],
            ticker,
            stocks_list["PurchasePrice"][row],
            stock.info["regularMarketPrice"],
            gain_loss(pp = stocks_list["PurchasePrice"][row], cp = stock.info["regularMarketPrice"]),
            growth(pp = stocks_list["PurchasePrice"][row], cp = stock.info["regularMarketPrice"]),
            annual_return(pd = stocks_list["PurchaseDate"][row], pp = stocks_list["PurchasePrice"][row], cp = stock.info["regularMarketPrice"]),
            stock.info["regularMarketPrice"]-stocks_list["PurchasePrice"][row],
            round(stock.info["volume"]/100000,2),
            round(stock.info["marketCap"]/10000000,2),
            stock.info["fiftyTwoWeekHigh"],
            stock.info["fiftyTwoWeekLow"],
            stocks_list["PurchaseDate"][row].strftime("%d/%m/%Y"),
            stocks_list["Expectation"][row],
        ], index=my_columns),
    ignore_index=True)

In [169]:
final_dataframe

,holder,qty,stock,purchase_price,current_price,per_gain_loss (%),per_growth (%),annual_return (%),tot_gain_loss,volume (lk),mark_cap (cr),week52high,week52low,purchase_date,expectance
0,Ridhi,100,RCOM.BO,1.75,3.24,85.14,185.14,85.14,1.49,437.49,889.14,4.33,0.62,20/05/2021,300


In [61]:
print(type(stocks_list["PurchaseDate"][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [65]:
import datetime
pd = stocks_list["PurchaseDate"][0]
cd = datetime.datetime.now()

(cd-pd).days

19

In [67]:
round(stock.info["volume"]/100000,2)

424.96

In [130]:
pip install XlsxWriter

Note: you may need to restart the kernel to use updated packages.


In [145]:
import xlsxwriter
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)


writer.sheets['Recommended Trades'].write('A1', 'Ticker')
writer.sheets['Recommended Trades'].write('B1', 'Price')
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization')
writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy')
writer.sheets['Recommended Trades'].set_column('A:A', 20)
writer.sheets['Recommended Trades'].set_column('B:B', 20)
writer.sheets['Recommended Trades'].set_column('C:C', 20)
writer.sheets['Recommended Trades'].set_column('D:D', 20)

    

writer.save()

writer.close()

In [140]:
import xlsxwriter

writer = pd.ExcelWriter("recommendedtrades.xlsx")

final_dataframe.to_excel(writer)



In [142]:
import xlsxwriter

workbook = xlsxwriter.Workbook('hello.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'Hello world')

workbook.close()

In [170]:
import pandas as pd

# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})


# filename with current date
filename = datetime.datetime.now().strftime("%d%m%Y")+'.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
final_dataframe.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [161]:
datetime.datetime.now().strftime("%d%m%Y")

'08062021'

In [163]:

filename = datetime.datetime.now().strftime("%d%m%Y")+'.xlsx'
filename

'08062021.xlsx'